In [14]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_excel('auto/data/1_中國大陸_進出口總值(美元).xlsx', index_col = 0)
# df = df[df['category_x'] == '進口總值(含復進口)']
indexs = df['date'].tolist() 
label_im = ['進口總值(含復進口)']
# data_im_indexs = df.iloc[:, 1].tolist()  # Convert to list
data_im_values = df['import_美元'].tolist()  # Convert to list

# df = pd.read_excel('auto/data/1_中國大陸_進出口總值(美元).xlsx', index_col = 0)
# df = df[df['category_y'] == '出口總值(含復出口)']
label_ex = ['出口總值(含復出口)']
# data_ex_indexs = df.iloc[:, 1].tolist()  # Convert to list
data_ex_values = df['export_美元'].tolist()  # Convert to list

In [28]:
df['年'] = df['date'].apply(lambda x: x.split('年')[0])
df['月'] = df['date'].apply(lambda x: x.split('年')[1])

In [32]:
def convert_roc_to_gregorian(roc_date):
    # Split year and month
    year_str, month_str = roc_date.split('年')
    year = int(year_str)
    month = int(month_str.replace('月', ''))
    
    # Convert ROC year to Gregorian year
    gregorian_year = year + 1911
    
    # Return as formatted string
    return f"{gregorian_year}-{month:02d}"

# Apply the conversion to the dataframe
df['gregorian_date'] = df['date'].apply(convert_roc_to_gregorian)

In [33]:
df.sort_values('gregorian_date')

,category_x,date,中文貨名_x,國家_x,import_美元,category_y,中文貨名_y,國家_y,export_美元,年,月,gregorian_date
160,NaN,92年3月,NaN,NaN,NaN,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,0,92,3月,2003-03
161,NaN,92年4月,NaN,NaN,NaN,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,0,92,4月,2003-04
162,進口總值(含復進口),92年6月,可互換之搪孔或拉孔工具，工具機用,中國大陸,0.0,NaN,NaN,NaN,NaN,92,6月,2003-06
163,進口總值(含復進口),92年7月,可互換之搪孔或拉孔工具，工具機用,中國大陸,0.0,NaN,NaN,NaN,NaN,92,7月,2003-07
164,進口總值(含復進口),92年9月,可互換之搪孔或拉孔工具，工具機用,中國大陸,0.0,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,1,92,9月,2003-09
...,...,...,...,...,...,...,...,...,...,...,...,...
155,進口總值(含復進口),112年9月,可互換之搪孔或拉孔工具，工具機用,中國大陸,8.0,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,553,112,9月,2023-09
144,進口總值(含復進口),112年10月,可互換之搪孔或拉孔工具，工具機用,中國大陸,3.0,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,552,112,10月,2023-10
145,進口總值(含復進口),112年11月,可互換之搪孔或拉孔工具，工具機用,中國大陸,20.0,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,882,112,11月,2023-11
146,進口總值(含復進口),112年12月,可互換之搪孔或拉孔工具，工具機用,中國大陸,5.0,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,868,112,12月,2023-12


In [19]:
data_im_values[2] == float('nan')

False

In [20]:
np.isnan(data_im_values[2])

True

In [18]:
type(data_im_values[2])

float

In [4]:
data_ex_values

[]

In [3]:
data_im_values

['0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '4',
 '3',
 '12',
 '20',
 '2',
 '0',
 '3',
 '5',
 '3',
 '4',
 '1',
 '1',
 '1',
 '3',
 '13',
 '3',
 '7',
 '4',
 '1',
 '1',
 '1',
 '17',
 '19',
 '26',
 '29',
 '5',
 '7',
 '7',
 '9',
 '2',
 '18',
 '26',
 '5',
 '11',
 '18',
 '20',
 '17',
 '20',
 '13',
 '23',
 '11',
 '6',
 '8',
 '8',
 '0',
 '1',
 '8',
 '14',
 '13',
 '18',
 '2',
 '28',
 '52',
 '23',
 '4',
 '19',
 '21',
 '8',
 '15',
 '2',
 '11',
 '24',
 '3',
 '2',
 '22',
 '3',
 '37',
 '21',
 '9',
 '23',
 '38',
 '47',
 '172',
 '43',
 '17',
 '8',
 '9',
 '6',
 '5',
 '4',
 '7',
 '8',
 '2',
 '3',
 '4',
 '4',
 '2',
 '14',
 '12',
 '11',
 '7',
 '6',
 '5',
 '11',
 '5',
 '12',
 '9',
 '11',
 '3',
 '8',
 '0',
 '7',
 '0',
 '8',
 '11',
 '0',
 '6',
 '1',
 '0',
 '0',
 '15',
 '5',
 '3',
 '4',
 '2',
 '9',
 '0',
 '0',
 '12',
 '20',
 '2',
 '5',
 '5',
 '13',
 '0',
 '1',
 '11',
 '6',
 '0',
 '11',
 '5',
 '2',
 '0',
 '0',
 '3',
 '6',
 '110',
 '2',
 '10',
 '5',
 '6',
 '11',
 '3',
 '27',
 '1',
 '23',
 '12',
 '27',
 '265',
 '6